In [ ]:
import json
import pandas as pd
from dateutil import parser
from math import pi
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Div
from bokeh.layouts import layout


events=[]
interactionEvents=[]
userID=0
time_modules=[0,0,0,0]
count_modules=[0,0,0,0]
time_avg=[0,0,0,0]
fn_string="def "
module_string = "import "
fn_count_list=[]
module_count_list=[]
error_count_list=[]
run_list=[]
files=['Module_1_variables/PythonBasicProgramming_20.ipynb',
       'Module_2_control_flow/PythonBasicProgramming_40.ipynb',
       'Module_3_code_organization/PythonBasicProgramming_60.ipynb', 
       'Module_4_basic_plotting/PythonBasicProgramming_80.ipynb']
modules = ["Variables", "Control flow", "Code Organization", "Basic Plotting"]

#Read the LogUI logs
with open('/srv/data/logui.log', mode= "r", encoding= "utf-8") as f:
    events = json.loads(f.read())

#Function to return module from logui logs    
def return_module(cond):
    if cond=="1":
        return "Variables"
    elif cond=="2":
        return "Control flow"
    elif cond=="3":
        return "Code Organization"
    elif cond=="4":
        return "Basic Plotting"

#Create a list with all events    
for event in events:
    if event['eventType']=='interactionEvent':
        value={'type':event['eventDetails']['type'], 
               'name':event['eventDetails']['name'], 
               'sessionID':event['sessionID'],
               'datetime':parser.parse(event['timestamps']['eventTimestamp']).replace(microsecond=0),
               'userID':event['applicationSpecificData']['userID'],               
               'module':return_module(event['applicationSpecificData']['condition'])
              }
        interactionEvents.append(value)

#Create a pandas dataframe and filter events by userID
df_events = pd.DataFrame(interactionEvents) 
user_events = df_events[df_events['userID']==userID]

#Filter events by type and aggregate each type
copy_events= user_events[user_events['name']=="CODE_BLOCK_COPY"]
copy_events_g = copy_events.groupby(['module']).agg({'name': ['count']})

paste_events= user_events[user_events['name']=="CODE_BLOCK_PASTE"]
paste_events_g = paste_events.groupby(['module']).agg({'name': ['count']})

keypress_events= user_events[user_events['name']=="CODE_BLOCK_KEYDOWN"]
keypress_events_g = keypress_events.groupby(['module']).agg({'name': ['count']})

##Keypress dashboard
source=ColumnDataSource(keypress_events_g)
module_list = source.data['module'].tolist()
p_keys = figure(x_range=module_list, plot_width=750, plot_height=750)

p_keys.vbar(x='module', top='name_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Number of keys pressed', '@name_count'),
]
p_keys.add_tools(hover)
p_keys.xaxis.major_label_orientation = pi/3
p_keys.title.text ='Keypresses per module'
p_keys.xaxis.axis_label = 'Module'
p_keys.yaxis.axis_label = 'Number of keys pressed'
p_keys.title.align = "center"

##Copy events dashboard
source=ColumnDataSource(copy_events_g)
module_list = source.data['module'].tolist()
p_copy = figure(x_range=module_list, plot_width=750, plot_height=750)
p_copy.vbar(x='module', top='name_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Copy events', '@name_count'),
]
p_copy.add_tools(hover)
p_copy.xaxis.major_label_orientation = pi/3
p_copy.title.text ='Copy events per module'
p_copy.xaxis.axis_label = 'Module'
p_copy.yaxis.axis_label = 'Number of copy events'
p_copy.title.align = "center"

##Paste events dashboard
source=ColumnDataSource(paste_events_g)
module_list = source.data['module'].tolist()
p_paste = figure(x_range=module_list, plot_width=750, plot_height=750)
p_paste.vbar(x='module', top='name_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Paste events', '@name_count'),
]
p_paste.add_tools(hover)
p_paste.xaxis.major_label_orientation = pi/3
p_paste.title.text ='Paste events per module'
p_paste.xaxis.axis_label = 'Module'
p_paste.yaxis.axis_label = 'Number of paste events'
p_paste.title.align = "center"

#Filter events by cell focus and cell blur and then compute the time spent on each cell
focus_array = ['CELL_FOCUS', 'CELL_BLUR']
focus_events = user_events.loc[user_events['name'].isin(focus_array)]

for i in range(len(focus_events)-1):
    if focus_events.iloc[i, 0]=="focusin" and focus_events.iloc[i+1,3]!="focusin":
        for j in range(i+1,len(focus_events)):
            if focus_events.iloc[j, 0]=="focusout" and focus_events.iloc[i,2]==focus_events.iloc[j,2] :
                time_on_cell=focus_events.iloc[j,3]-focus_events.iloc[i,3]
                if(time_on_cell.total_seconds()>=0):
                    break

    for ind in range(len(modules)):
        if(focus_events.iloc[i,5]==modules[ind]):
            time_modules[ind]= time_modules[ind]+time_on_cell.total_seconds()
            count_modules[ind]=count_modules[ind]+1
            
for ind in range(len(time_modules)):
    if(count_modules[ind]>0):
        time_avg[ind]=time_modules[ind]/count_modules[ind]
        
#Dashboard for time spent on cell
source = ColumnDataSource(data=dict(
    time_avg=time_avg,
    module=modules
))
p_focus = figure(x_range=modules, plot_width=750, plot_height=750)
p_focus.vbar(x='module', top='time_avg', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Average time spent on a cell', '@time_avg seconds'),
]
p_focus.add_tools(hover)
p_focus.xaxis.major_label_orientation = pi/3
p_focus.title.align = "center"
p_focus.title.text ='Average time spent on a cell per module (in seconds)'
p_focus.xaxis.axis_label = 'Module'
p_focus.yaxis.axis_label = 'Average time spent on a cell'

#Read files and compute parameter values per module
for file in files:
    cells_all=[]
    with open(file, mode= "r", encoding= "utf-8") as f:
        myfile = json.loads(f.read())
        cells_all.extend(myfile['cells'])

    fn_count=0
    module_count=0
    error_count=0
    execution_count=[]
  
    for cell in cells_all:
        if cell['cell_type']=="code" and len(cell['source'])!=0:
            if (cell['execution_count']):
                execution_count.append(cell['execution_count'])
            source=cell['source']
            outputs = cell['outputs']
            for line in source:
                if (line.find(fn_string)!=-1):
                    fn_count = fn_count +1
                if (line.find(module_string)!=-1):
                    module_count = module_count +1    
            for output in outputs:
                if(output['output_type']=="error"):
                    error_count=error_count+1    

    runCount = max(execution_count)
    fn_count_list.append(fn_count)
    module_count_list.append(module_count)
    error_count_list.append(error_count)
    run_list.append(runCount)

#Functions per module dashboard
source = ColumnDataSource(data=dict(
    fn_count=fn_count_list,
    module_count=module_count_list,
    error_count=error_count_list,
    run_list=run_list,
    module=modules
))
p_fn = figure(x_range=modules, plot_height=750, plot_width=750)
p_fn.vbar(x='module', top='fn_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Function count', '@fn_count'),
]
p_fn.add_tools(hover)
p_fn.xaxis.major_label_orientation = pi/3
p_fn.title.text ='Functions defined per module'
p_fn.xaxis.axis_label = 'Module'
p_fn.yaxis.axis_label = 'Number of defined functions'
p_fn.title.align = "center"

#Module imports dashboard
p_mod = figure(x_range=modules, plot_height=750, plot_width=750)
p_mod.vbar(x='module', top='module_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Module Import count', '@module_count'),
]
p_mod.add_tools(hover)
p_mod.xaxis.major_label_orientation = pi/3
p_mod.title.text ='Modules imported per learning module'
p_mod.xaxis.axis_label = 'Module'
p_mod.yaxis.axis_label = 'Number of imported modules'
p_mod.title.align = "center"

#Errors dashboard
p_err = figure(x_range=modules, plot_height=750, plot_width=750)
p_err.vbar(x='module', top='error_count', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Error count', '@error_count'),
]
p_err.add_tools(hover)
p_err.xaxis.major_label_orientation = pi/3
p_err.title.text ='Errors per module'
p_err.xaxis.axis_label = 'Module'
p_err.yaxis.axis_label = 'Number of errors'
p_err.title.align = "center"

#Cell runs dashboard
p_run = figure(x_range=modules, plot_height=750, plot_width=750)
p_run.vbar(x='module', top='run_list', source=source, width=0.70, color="steelblue")
hover = HoverTool()
hover.tooltips=[
    ('Module', '@module'),
    ('Cell runs', '@run_list'),
]
p_run.add_tools(hover)
p_run.xaxis.major_label_orientation = pi/3
p_run.title.text ='Cell runs per module'
p_run.xaxis.axis_label = 'Module'
p_run.yaxis.axis_label = 'Number of cell runs'
p_run.title.align = "center"

output_file('CT_practices.html', title="CT practices dashboard")

#Define the header div
div_header = Div(text="""<h1>\tCT practices dashboard</h1>""", background="steelblue", width=1500, height=75, style={'color': 'white', 'padding-left':'30px'})

#CT practices
div_bit = Div(text="""<h2>Practice: Being Incremental and Iterative</h2>""", width=1500, height=50, style={'padding-left':'600px'})
div_td = Div(text="""<h2>Practice: Testing and Debugging</h2>""", width=1500, height=50, style={'padding-left':'600px'})
div_rr = Div(text="""<h2>Practice: Reusing and Remixing</h2>""", width=1500, height=50, style={'padding-left':'600px'})
div_am = Div(text="""<h2>Practice: Abstracting and Modularizing</h2>""", width=1500, height=50, style={'padding-left':'600px'})


#Define the action div - being incremental and iterative
div_action_bit = Div(text="""<h3><i>ACTION</i>: Spend more time on each cell and try out different solutions</h3>""", width=1500, height=50)

#Define the action div - testing and debugging
div_action_td = Div(text="""<h3><i>ACTION</i>: Test your code by running each cell and debug any errors that you get</h3>""", width=1500, height=50)

#Define the action div - reusing and remixing
div_action_rr = Div(text="""<h3><i>ACTION</i>: Reuse and remix existing code by copying and pasting </h3>""", width=1500, height=50)

#Define the action div - abstracting and modularizing
div_action_am = Div(text="""<h3><i>ACTION</i>: Use more functions and module imports to simplify your code</h3>""", width=1500, height=50)

#Set the dashboard layout and display it
layout_db = layout([
    [div_header],
    [div_bit],
    [p_keys, p_focus],
    [div_action_bit],
    [div_td],
    [p_run, p_err],
    [div_action_td],
    [div_rr],
    [p_copy, p_paste],
    [div_action_rr],
    [div_am],
    [p_fn, p_mod],
    [div_action_am]
])
show(layout_db) 